In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [12]:
df = pd.DataFrame()
for i in tqdm(range(400)):
    content = requests.get(f"https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={i}&region=2&room1=1&room2=1").content
    root = BeautifulSoup(content, "html.parser")
    cards = root.find("div", class_="_93444fe79c--wrapper--W0WqH")

    for car_card in cards.find_all("div", class_="_93444fe79c--card--ibP42 _93444fe79c--wide--gEKNN"):
        try:
            title = car_card.find("span", class_="_93444fe79c--color_primary_100--mNATk _93444fe79c--lineHeight_28px--whmWV _93444fe79c--fontWeight_bold--ePDnv _93444fe79c--fontSize_22px--viEqA _93444fe79c--display_block--pDAEx _93444fe79c--text--g9xAG _93444fe79c--text_letterSpacing__normal--xbqP6").text
            second_title = car_card.find("span", class_="_93444fe79c--color_black_100--kPHhJ _93444fe79c--lineHeight_22px--bnKK9 _93444fe79c--fontWeight_bold--ePDnv _93444fe79c--fontSize_16px--RB9YW _93444fe79c--display_block--pDAEx _93444fe79c--text--g9xAG _93444fe79c--text_letterSpacing__normal--xbqP6").text
            metro = car_card.find("div", class_="_93444fe79c--remoteness--q8IXp").text
            price = car_card.find("span", class_="_93444fe79c--color_black_100--kPHhJ _93444fe79c--lineHeight_28px--whmWV _93444fe79c--fontWeight_bold--ePDnv _93444fe79c--fontSize_22px--viEqA _93444fe79c--display_block--pDAEx _93444fe79c--text--g9xAG _93444fe79c--text_letterSpacing__normal--xbqP6").text
            df_concat = pd.DataFrame({'title':[title], 'second_title':[second_title], 'metro':[metro], 'price':[price]})
            df = pd.concat([df, df_concat])
        except:

In [13]:
df = df[(df['title'].apply(lambda x: x[-4:]) == 'этаж') & (df['title'].apply(lambda x: x[0].isdigit()))] 

In [17]:
df.isnull().sum()

metro     0
price     0
rooms     0
floor     0
floors    0
square    0
turned    0
dtype: int64

In [56]:
df.title

0     2-комн. кв., 61,31 м², 6/19 этаж
0     1-комн. кв., 39,35 м², 2/19 этаж
0     2-комн. кв., 60,6 м², 15/18 этаж
0      1-комн. кв., 37,4 м², 2/18 этаж
0        1-комн. кв., 37 м², 2/18 этаж
                   ...                
0        1-комн. кв., 35 м², 2/12 этаж
0       1-комн. кв., 54,3 м², 6/9 этаж
0    1-комн. кв., 37,35 м², 10/13 этаж
0     1-комн. кв., 35,03 м², 1/13 этаж
0       1-комн. кв., 42,1 м², 8/8 этаж
Name: title, Length: 7921, dtype: object

In [1]:
df['price'] = df['price'].apply(lambda x: x.replace('₽', '').replace(' ', '')).astype(int)
df['rooms'] = df['title'].apply(lambda x: x.split('-')[0]).astype(int)
df['floor'] = df['title'].apply(lambda x: x.split(' ')[-2].split('/')[0]).astype(int)
df['floors'] = df['title'].apply(lambda x: x.split(' ')[-2].split('/')[1]).astype(int)
df['square'] = df['title'].apply(lambda x: x.split(' ')[-4].replace(',', '.')).astype(float)
df['turned'] = df['second_title'].apply(lambda x: 1 if x.find('Сдан') != -1 else 0)

In [2]:
df['metro'] = df['metro'].apply(lambda x: (x.find('пешком') != -1)*1)

In [66]:
df.head()

,title,second_title,metro,price,rooms,floor,floors,square,turned
0,"2-комн. кв., 61,31 м², 6/19 этаж",Секция 10 • Сдача корпуса 1 кв. 2025,0,14629999,2,6,19,61.31,0
0,"1-комн. кв., 39,35 м², 2/19 этаж",Секция 16 • Сдача корпуса 1 кв. 2025,0,9799999,1,2,19,39.35,0
0,"2-комн. кв., 60,6 м², 15/18 этаж",Секция 12 • Сдан,1,15441588,2,15,18,60.60,1
0,"1-комн. кв., 37,4 м², 2/18 этаж",Секция 2 • Сдан,1,9350000,1,2,18,37.40,1
0,"1-комн. кв., 37 м², 2/18 этаж",Секция 1 • Сдача корпуса 4 кв. 2024,0,10239999,1,2,18,37.00,0


In [3]:
df.drop(['title', 'second_title'], axis=1, inplace=True)

In [75]:
df.to_csv('df_cian.csv', index=False)

In [15]:
df = pd.read_csv('df_cian.csv')

In [16]:
df

,metro,price,rooms,floor,floors,square,turned
0,0,14629999,2,6,19,61.31,0
1,0,9799999,1,2,19,39.35,0
2,1,15441588,2,15,18,60.60,1
3,1,9350000,1,2,18,37.40,1
4,0,10239999,1,2,18,37.00,0
...,...,...,...,...,...,...,...
7915,0,5859341,1,2,12,35.00,0
7916,1,23324180,1,6,9,54.30,1
7917,0,8779520,1,10,13,37.35,1
7918,0,6449160,1,1,13,35.03,0
